# Magnetotellurics

This object can be used to store magnetotelluric (MT) data - a natural-source geophysical method widely used for deep crustal investigations. Electromagntic data are provided in the frequency-domain as point source measurments of either impedances or apparent resistity/phase.

The following example shows how to generate an MT survey with associated data programmatically.

![mtSurvey](./images/mt_survey.png)

In [ ]:
import numpy as np
from geoh5py.workspace import Workspace
from geoh5py.objects.surveys.magnetotellurics import Magnetotellurics

# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the receiver locations on 2 lines, 60 m apart
x_loc, y_loc = np.meshgrid(np.linspace(-5, 5, 2), np.linspace(0., 20., 9))
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Create the survey from vertices
mt_survey = Magnetotellurics.create(workspace, vertices=vertices)

## Channels

User must define the frequencies at which the data are provided.

In [ ]:
mt_survey.channels = [1., 10., 100.]

## Data

The MT survey expects data to be grouped by component and in the same order as the input channels. The class method `add_frequency_data` streamlines the import of data from a nested dictionary. 

In [ ]:
# Create some simple data
data = {}
d_vec = (x_loc * y_loc).ravel() / 200.
for cc, component in enumerate(["Zxx (real)", "Zxx (imaginary)", "Zxy (real)", "Zyy (real)"]):
    comp_dict = {}
    for ff, freq in enumerate(mt_survey.channels):
        comp_dict[freq] = {"values": (cc+1.) * np.sin(ff * np.pi * d_vec)}
        
    data[component] = comp_dict
    
mt_survey.add_frequency_data(data)

**Note:** Not all components of the impedence tensor need to be provided. It is expected however that each component contains one data channels per frequency. The data channel supplied must be in the same order as defined by the `survey.channels`.

## Metadata

Along with the survey object itself, the metadata contains all the necessary information to define the geophysical experiment.

In [ ]:
mt_survey.metadata